# Задание 2. SQL
## 2.1 Очень усердные ученики.

### 2.1.1 Условие

Образовательные курсы состоят из различных уроков, каждый из которых состоит из нескольких маленьких заданий. Каждое такое маленькое задание называется "горошиной".

Назовём очень усердным учеником того пользователя, который хотя бы раз за текущий месяц правильно решил 20 горошин.

### 2.1.2 Задача

Дана таблица default.peas
Необходимо написать оптимальный запрос, который даст информацию о количестве очень усердных студентов.NB! Под усердным студентом мы понимаем студента, который правильно решил 20 задач за текущий месяц.

## 2.2 Оптимизация воронки

### 2.2.1 Условие

Образовательная платформа предлагает пройти студентам курсы по модели trial: студент может решить бесплатно лишь 30 горошин в день. Для неограниченного количества заданий в определенной дисциплине студенту необходимо приобрести полный доступ. Команда провела эксперимент, где был протестирован новый экран оплаты.

### 2.2.2 Задача

Дана таблицы: default.peas, default.studs и default.final_project_check.
Необходимо в одном запросе выгрузить следующую информацию о группах пользователей:

ARPU 

ARPAU 

CR в покупку 

СR активного пользователя в покупку 

CR пользователя из активности по математике (subject = ’math’) в покупку курса по математике

ARPU считается относительно всех пользователей, попавших в группы.
Активным считается пользователь, за все время решивший больше 10 задач правильно в любых дисциплинах.
Активным по математике считается пользователь, за все время решивший 2 или больше задач правильно по математике.

In [1]:
import pandahouse as ph

In [2]:
connection_default = {'host': 'https://clickhouse.lab.karpov.courses',
                      'database':'default',
                      'user':'student', 
                      'password':'dpo_python_2020'
                     }
## Объявляем переменную с данными

Первое задание:

In [3]:
q = '''
    SELECT COUNT(st_id) AS NumberOfGoodStudents
    FROM
        (
        SELECT st_id,COUNT(correct)
        FROM peas
        WHERE 
            correct = 1
        GROUP BY 
            st_id
        HAVING 
            COUNT(correct)>19
        )

    '''

q_task1 = ph.read_clickhouse(query=q, connection=connection_default)
q_task1

,NumberOfGoodStudents
0,136


Второе задание:

In [4]:
q2 = '''
    SELECT
        arpu.test_grp AS test_grp,
        arpu.arpu AS arpu,
        arpau.arpau AS arpau,
        cr.cr AS cr,
        Cr_active.cr_active AS cr_active,
        cr_math.cr_math AS cr_math
    FROM (
        SELECT test_grp, SUM(money) / uniqExact(st_id) AS arpu
        FROM (
            SELECT *
            FROM default.studs AS a
            FULL JOIN default.final_project_check AS b ON a.st_id = b.st_id
        )
        GROUP BY test_grp
    ) AS arpu
    JOIN (
        SELECT c.test_grp as test_grp, SUM(b.money) / count(DISTINCT a.st_id) AS arpau
        FROM (
            SELECT st_id, SUM(correct) AS SumCorrect
            FROM default.peas
            GROUP BY st_id
            HAVING SumCorrect > 10
        ) AS a
        LEFT JOIN default.final_project_check AS b ON a.st_id = b.st_id
        LEFT JOIN default.studs AS c ON a.st_id = c.st_id
        GROUP BY c.test_grp
    ) AS arpau ON arpu.test_grp = arpau.test_grp
    JOIN (
        SELECT c.test_grp as test_grp, (count(DISTINCT b.st_id) / count(DISTINCT a.st_id)) AS cr_active
        FROM (
            SELECT st_id, SUM(correct) AS SumCorrect
            FROM default.peas
            GROUP BY st_id
            HAVING SumCorrect > 10
        ) AS a
        LEFT JOIN default.final_project_check AS b ON a.st_id = b.st_id
        LEFT JOIN default.studs AS c ON a.st_id = c.st_id
        GROUP BY c.test_grp
    ) AS Cr_active ON arpu.test_grp = Cr_active.test_grp
    JOIN (
        SELECT a.test_grp as test_grp, count(DISTINCT b.st_id) / count(DISTINCT a.st_id) AS cr
        FROM (
            SELECT *
            FROM default.studs AS a
            FULL JOIN default.final_project_check AS b ON a.st_id = b.st_id
            FULL JOIN default.peas AS c ON a.st_id = c.st_id
        )
        GROUP BY a.test_grp
    ) AS cr ON arpu.test_grp = cr.test_grp
    JOIN (
        SELECT a.test_grp as test_grp, uniqIf(a.st_id, b.money > 0 AND c.st_m_active >= 2 AND b.subject = 'Math') / uniqIf(a.st_id, c.st_m_active >= 2) AS cr_math
        FROM default.studs AS a
        LEFT JOIN default.final_project_check AS b ON a.st_id = b.st_id
        LEFT JOIN (
            SELECT st_id, SUM(correct) AS st_active, sumIf(correct, subject = 'Math') AS st_m_active
            FROM default.peas
            GROUP BY st_id
        ) AS c ON a.st_id = c.st_id
        GROUP BY a.test_grp
    ) AS cr_math ON arpu.test_grp = cr_math.test_grp
    '''

q_task2 = ph.read_clickhouse(query=q2, connection=connection_default)
q_task2

,test_grp,arpu,arpau,cr,cr_active,cr_math
0,control,4540.983607,10393.700787,0.052459,0.118110,0.061224
1,pilot,11508.474576,29739.583333,0.111864,0.270833,0.095238
